# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
import drawsvg as draw
from typing import Literal

from python import constants, functions

# Testing and Debugging

In [17]:
def round_min(value, rounding, min_distance=0.1):
    rounded = np.floor(value / rounding) * rounding

    if value - rounded < rounding * min_distance:
        rounded -= rounding

    return int(rounded)

def round_max(value, rounding, min_distance=0.1):
    rounded = np.ceil(value / rounding) * rounding

    if rounded - value < rounding * min_distance:
        rounded += rounding

    return int(rounded)

class svg(html_tag):
    pass

class g(html_tag):
    pass

class text(html_tag):
    pass

class path(html_tag):
    pass

class circle(html_tag):
    pass

class rect(html_tag):
    pass

def df_to_svg(
        data: pd.DataFrame, 
        x_col: str, 
        y_col: str,
        chart_type: str = 'scatter',
        width: int = 500, 
        height: int = 300,
        x_tick_spacing: int = 50,
        y_tick_spacing: int = 50
):

    # margin x and y from edges of visual
    m_top, m_bottom, m_left, m_right = 10, 50, 60, 10
    # plot width and height
    P_x, P_y = width - m_left - m_right, height - m_top - m_bottom
    # tick margin (distance between axes and tick label)
    m_tick = 5

    x_min, x_max = round_min(data[x_col].min(), rounding=x_tick_spacing), round_max(data[x_col].max(), rounding=x_tick_spacing)
    y_min, y_max = round_min(data[y_col].min(), rounding=y_tick_spacing), round_max(data[y_col].max(), rounding=y_tick_spacing)

    x_ticks = [(i * x_tick_spacing) + x_min for i in range(1, int((x_max - x_min) / x_tick_spacing))]
    y_ticks = [(i * y_tick_spacing) + y_min for i in range(1, int((y_max - y_min) / y_tick_spacing))]

    if chart_type == 'bar':
        y_max = max(abs(y_min), abs(y_max))
        
        if y_min >= 0:
            y_min = 0
        else:
            y_min = -y_max

        y_ticks = [(i * y_tick_spacing) + y_min for i in range(1, int((y_max - y_min) / y_tick_spacing))]

    d_svg = svg(
        xmlns='http://www.w3.org/2000/svg',
        width=width,
        height=height,
        viewBox=f'0 0 {width} {height}'
    )

    outer_group = g(
        **{
            'font-family':'Arial',
            'font-size':10,
            'text-anchor':'middle',
            'dominant-baseline':'hanging'
        }
    )
    d_svg.add(outer_group)

    border = draw.Lines(0, 0, width, 0, width, height, 0, height, close=True)
    border_path = path(d=border.args['d'], fill='white', _id='border')
    outer_group.add(border_path)

    

    xlabel_group = g(
        **{
            'font-family':'Arial',
            'font-size':10,
            'text-anchor':'middle',
            'dominant-baseline':'hanging'
        }
    )

    grid_path_d = []
    for xtick in x_ticks:
        x = round(m_left + (xtick - x_min) / (x_max - x_min) * P_x, 3)

        gridline = draw.Line(x, m_top, x, height - m_bottom) 
        grid_path_d.append(gridline.args['d'])

        xtick_label = text(
            xtick,
            x=x,
            y=height - m_bottom + m_tick
        )
        xlabel_group.add(xtick_label)

    outer_group.add(xlabel_group)

    ylabel_group = g(
        **{
            'font-family':'Arial',
            'font-size':10,
            'text-anchor':'end',
            'dominant-baseline':'middle'
        }
    )
    zero_grid_path = False
    zero_ytick = False
    for ytick in y_ticks:
        y = round((height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, 3)

        gridline = draw.Line(m_left, y, width - m_right, y)
        grid_path_d.append(gridline.args['d'])

        ytick_label = text(
            ytick,
            x=m_left - m_tick,
            y=y
        )
        ylabel_group.add(ytick_label)


        if ytick == 0:
            zero_grid_path = path(d=gridline.args['d'], stroke='black')
            zero_ytick = y

    outer_group.add(ylabel_group)

    grid_path = path(
        d=' '.join(grid_path_d),
        stroke='lightgrey'
    )
    outer_group.add(grid_path)
    if zero_grid_path:
        outer_group.add(zero_grid_path)

    axis_title_group = g(
        **{
            'font-size':16
        }
    )
    outer_group.add(axis_title_group)
    xlabel = text(
        x_col,
        x=(P_x / 2 + m_left),
        y=(height - (m_bottom / 2))
    )
    ylabel = text(
        y_col,
        x=10,
        y=(P_y / 2 + m_top),
        transform=f'rotate(-90, {10}, {P_y / 2 + m_top})'
    )

    axis_title_group.add([xlabel, ylabel])
    axes = draw.Lines(m_left, m_top, width - m_right, m_top, width - m_right, height - m_bottom, m_left, height - m_bottom, close=True)
    axes_path = path(d=axes.args['d'], fill='none', stroke='black', _id='axes')
    outer_group.add(axes_path)

    points = []
    circles_group = g(
        **{
            'stroke':'black',
            'stroke-width':1.5
        }
    )
    bars_group = g(stroke='black')
    for index, row in data.iterrows():
        v_x = round(m_left + ((row[x_col] - x_min) / (x_max - x_min) * (P_x)), 3)
        v_y = round((height - m_bottom) - ((row[y_col] - y_min) / (y_max - y_min) * (P_y)), 3)

        bar_y = v_y if row[y_col] > 0 else zero_ytick
        bar_height = zero_ytick - v_y if row[y_col] > 0 else v_y - zero_ytick
        bar_width = 25

        bars_group.add(
            rect(
                x=v_x - bar_width / 2,
                y=bar_y,
                width=bar_width,
                height=bar_height,
                fill=constants.COLOR_DICT[row['Team'].lower()]
            )
        )
        circles_group.add(circle(cx=v_x, cy=v_y, r=4, fill=constants.COLOR_DICT[row['Team'].lower()]))
        
        points.extend([v_x, v_y])

    if chart_type == 'line':
        line = draw.Lines(*points)
        line_path = path(d=line.args['d'], fill='none', stroke='black')
        outer_group.add(line_path)

    if chart_type in ['scatter','line']:
        outer_group.add(circles_group)

    if chart_type == 'bar':
        outer_group.add(bars_group)

    return d_svg

seasons = [functions.summary_table(constants.GAME_DATA, year=year) for year in constants.YEARS]
seasons_df = pd.concat(seasons)
seasons_df = seasons_df.loc[seasons_df['Team'] == 'Noah']

data = functions.summary_table(constants.GAME_DATA, year=2021)

clean_svg = df_to_svg(
    data=seasons_df,
    x_col='Year',
    y_col='Luck Score',
    chart_type='bar',
    x_tick_spacing=1,
    y_tick_spacing=2
)

print(clean_svg)

<svg height="300" viewBox="0 0 500 300" width="500" xmlns="http://www.w3.org/2000/svg">
  <g dominant-baseline="hanging" font-family="Arial" font-size="10" text-anchor="middle">
    <path d="M0,0 L500,0 L500,300 L0,300 Z" fill="white" id="border"></path>
    <g dominant-baseline="hanging" font-family="Arial" font-size="10" text-anchor="middle">
      <text x="113.75" y="255">2018</text>
      <text x="167.5" y="255">2019</text>
      <text x="221.25" y="255">2020</text>
      <text x="275.0" y="255">2021</text>
      <text x="328.75" y="255">2022</text>
      <text x="382.5" y="255">2023</text>
      <text x="436.25" y="255">2024</text>
    </g>
    <g dominant-baseline="middle" font-family="Arial" font-size="10" text-anchor="end">
      <text x="55" y="226.0">-8</text>
      <text x="55" y="202.0">-6</text>
      <text x="55" y="178.0">-4</text>
      <text x="55" y="154.0">-2</text>
      <text x="55" y="130.0">0</text>
      <text x="55" y="106.0">2</text>
      <text x="55" y="82.0

In [19]:
def summary_table(data: pd.DataFrame, year: int, week: int = None) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    champ_week = data.loc[data['Year'] == year, 'Week'].max()
    champ = data.loc[(data['Year'] == year) & (data['Week'] == champ_week) & (data['Win'] == 1), 'Team'].item()

    league_pfpg = round(temp['Score'].mean(), 2)

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        pfpg = round(temp_team['Score'].mean(), 2)
        pfpg_plus = int(pfpg / league_pfpg * 100)
        pa = round(temp_team['Opp Score'].sum(), 2)
        papg = round(temp_team['Opp Score'].mean(), 2)
        papg_plus = int(papg / league_pfpg * 100)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()

        champ_flag = int(champ == team)

        weekly_standings.append(
            {
                'Team':team,
                'Wins':wins,
                'Losses':losses,
                'Record':record,
                'Points For':pf,
                'Points Against':pa,
                'PF/G':pfpg,
                'PF/G+':pfpg_plus,
                'PA/G':papg,
                'PA/G+':papg_plus,
                'Avg Margin':avg_margin,
                'Luck Score':luck_score,
                'Champ Flag':champ_flag
            }
        )

    weekly_standings = pd.DataFrame(weekly_standings)
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    weekly_standings['Year'] = year

    return weekly_standings

In [20]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

data

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [24]:
def df_to_table(
        data: pd.DataFrame,
        custom_columns: list[str] = None,
        row_id_columns: list[str] = None,
        table_id: str = None,
        champ_class: bool = False,
        footer_data: pd.DataFrame = None
) -> table:
    '''
    Converts a pandas DataFrame into an HTML table with optional styling.

    If dataframe is longer than 20 rows, table will be wrapped in a "scroll-table" div.

    Parameters
    ----------
    data : pd.DataFrame
        Dataframe to be converted.

    custom_columns : list[str], default None
        Columns to be displayed in the HTML table. If set to None, all columns in DataFrame will be used.

    row_id_columns : list[str], default None
        ID to be assigned to each tr tag in the tbody.
        Accepts a list of columns, will concatenate them starting with 'row' and separating with '-'.
        If set to None, no IDs will be assigned.

    table_id : str, default None
        ID to be assigned to the main table tag. If set to None, no ID will be given.

    champ_class : bool, default False

    table_footer : str {'totals','averages','both'}

    Returns
    -------
    dominate.table or dominate.div
        Return a dominate HTML object. Can be either a table or div
    '''
    t = table()
    head = thead()
    body = tbody()
    foot = tfoot()

    records = data.to_dict('records')

    columns = data.columns
    if custom_columns:
        columns = custom_columns

    column_row = tr()
    for column in columns:
        column_row.add(th(column, _class=str(column).lower().replace(' ','-')))
    head.add(column_row)
    
    for record in records:
        data_row = tr(__pretty=False)
        if champ_class:
            if record['Champ Flag'] == 1:
                data_row['class'] = 'champ'
        if row_id_columns:
            row_id = 'row'
            for column in row_id_columns:
                row_id += ('-' + str(record[column]).lower().replace(' ',''))
            data_row['id'] = row_id

        for column in columns:
            data_row.add(td(record[column], _class=str(column).lower().replace(' ','-')))
        body.add(data_row)

    if footer_data:
        footer_records = footer_data.to_dict('records')

        for record in footer_records:
            footer_row = tr(__pretty=False)
            for column in columns:
                footer_row.add(th(record[column], _class=str(column).lower().replace(' ','-')))
            foot.add(footer_row)

    t.add(head)
    t.add(body)
    if footer_data:
        t.add(foot)
    if table_id:
        t['id'] = table_id

    if len(data) > 20:
        scroll_div = div(_class='scroll-table')
        scroll_div.add(t)
        return scroll_div

    return t

In [31]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = [{
    'Year':'Total',
    'Wins':round(data['Wins'].mean(), 2),
    'Losses':round(data['Losses'].mean(), 2),
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}]

pd.DataFrame(averages)

,Year,Wins,Losses,Ranking,Points For,Points Against,PF/G,PF/G+,Avg Margin,Luck Score
0,Total,6.43,7.14,6.43,1573.72,1552.79,115.74,99.71,1.24,-0.71
